In [ ]:
# pip install -U scikit-learn
# pip install wandb -qU
# pip uninstall wandb


In [ ]:
!pip install wandb -qU

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import wandb
from keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn import metrics


In [ ]:
!wandb login 494428cc53b5c21da594f4fc75035d136c63a93c

# Question 1

In [ ]:
wandb.init(project="CS6910 - Assignment 1", name="Question1")

In [ ]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Define class names
className_images = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
trainingData_length=len(train_labels)
total_class=max(train_labels)+1

# print(trainingData_length)
# print(y)

#plotting images with its labels
plt.figure(figsize=(5, 5))
for i in range(total_class):
    #subplot - used so that we can have many images in one image
    # 1st row,2nd col, 3rd index of curr
    plt.subplot(3, 4, i + 1)
    sample_array = np.where(train_labels == i)
    label_index=sample_array[0][0]
    plt.imshow(train_images[label_index], cmap='gray')
    plt.title(className_images[i])
    plt.axis('off')
    wandb.log({"Question1": [wandb.Image(train_images[label_index], caption=className_images[i])]})

plt.show()

In [ ]:
wandb.finish()

# MultiLayerPerceptron class

In [ ]:
class MultiLayerPerceptron():

    def __init__(
          self,
          train_data,
          train_labels,
          val_data,
          val_labels,
          layer_sizes,
          initialization,
          loss_act_func,
          batch_size=4,
          epochs=1,
          learning_rate=0.1,
          weight_decay=0,
          optimizer="sgd",
          momentum=0.5,
          beta=0.5,
          beta1=0.5,
          beta2=0.5,
          epsilon=0.000001
          ):

        self.parameters = {}
        self.gradients = {}
        self.train_data = train_data
        self.train_labels = train_labels
        self.val_data=val_data
        self.val_labels=val_labels
        self.no_of_samples = len(train_data)
        self.no_of_features = train_data.shape[1] #size of column of train_data
        self.no_of_classes = 10 #size of column of train_labels
        self.layer_sizes = layer_sizes
        self.batch_size=batch_size
        # self.act_func=act_func
        self.epochs=epochs
        self.learning_rate=learning_rate
        self.weight_decay=weight_decay
        self.optimizer=optimizer
        self.loss_act_func=loss_act_func
        self.initialization = initialization
        self.momentum=momentum
        self.beta=beta
        self.beta1=beta1
        self.beta2=beta2
        self.epsilon=epsilon

        self.A = {}
        self.H = {}

    def init_weights(self):
      if self.initialization=="random":
        # print("init random")
        for i in range(len(self.layer_sizes) - 1):
            #input layer(n) and 1st hidden layer(m) will have weight vector w1->n*m and bias b1->1*m
            #init W to random values
            self.parameters[f'w_{i+1}'] = np.random.randn(self.layer_sizes[i], self.layer_sizes[i+1])
            #init B to 0
            self.parameters[f'b_{i+1}'] = np.random.randn(1,self.layer_sizes[i+1])

      if self.initialization=="Xavier":
        print("inint Xavier")
        for i in range(len(self.layer_sizes) - 1):
          x_fact = np.sqrt(6 / (self.layer_sizes[i] + self.layer_sizes[i + 1]))
          self.parameters[f'w_{i+1}'] = np.random.randn(self.layer_sizes[i], self.layer_sizes[i + 1]) * x_fact
          self.parameters[f'b_{i+1}'] = np.random.randn(1,self.layer_sizes[i+1]) * x_fact

    def softmax(self, x):
      max_x = np.max(x, axis=1, keepdims=True)
      exps = np.exp(x - max_x)
      return exps / np.sum(exps, axis=1, keepdims=True)

    def softmax_d(self,x):
      softmax_x = self.softmax(x)
      return softmax_x*(1-softmax_x)

    def feed_forwards(self, input_data):

        # self.H[0]=input_data
        for i in range(len(self.layer_sizes) - 1): # i -> 0-3
            # if(i==0) :#input is our images
            if(i==0):
              preactivation=np.dot(input_data , self.parameters[f'w_{i+1}']) + self.parameters[f'b_{i+1}']
            else:
              preactivation = np.dot(self.H[i], self.parameters[f'w_{i+1}']) + self.parameters[f'b_{i+1}']
            self.A[i+1]=preactivation

            if i < len(self.layer_sizes) - 2 : # i < 3
                activation = self.loss_act_func.activation_func(self.A[i+1])
            else :
                activation=self.softmax(self.A[i+1])
            self.H[i+1]=activation

        return self.H[len(self.layer_sizes)-1]


    def back_prop(self, input_data, true_label,pred_label):
        # print("in back")
        if ((self.loss_act_func).get_name()=="cross_entropy"):
          # print("cross_entropy")
          dL_dA = pred_label - true_label
        elif ((self.loss_act_func).get_name()=="mean_squared_error"):
          dL_dA = ((pred_label - true_label) * self.softmax_d(self.A[len(self.layer_sizes)-1])) / true_label.shape[0]

        for i in range(len(self.layer_sizes) - 1, 1, -1): # i->4 3 2

            self.gradients[f'w_{i}']=np.dot(self.H[i-1].T, dL_dA)
            self.gradients[f'b_{i}']=np.sum(dL_dA , axis=0,keepdims=True)

            dL_dH=np.dot(dL_dA, (self.parameters[f'w_{i}']).T )
            dL_dA = dL_dH * self.loss_act_func.derivation_of_activation(self.A[i-1])

        self.gradients[f'w_{1}']=np.dot(input_data.T, dL_dA )
        self.gradients[f'b_{1}']=np.sum(dL_dA , axis=0,keepdims=True)

    def optimizer_func(self):

        if(self.optimizer=="sgd"):
            #this is batch gradient descent or SGD
            # no_of_batches = self.no_of_samples // batch_size
            for i in range(self.epochs):
                for j in range(0,self.no_of_samples,self.batch_size):
                    start = j
                    end = start + self.batch_size
                    Y_train_labels = self.train_labels[start:end]
                    X_train_images = self.train_data[start:end]

                    pred_labels=self.feed_forwards(X_train_images)
                    # pred_labels=self.fprg(X_train_images)
                    self.back_prop(X_train_images, Y_train_labels,pred_labels)

                    for k in range(len(self.layer_sizes) - 1):# 0->3
                        self.gradients[f'w_{k+1}']=self.gradients[f'w_{k+1}']/self.batch_size
                        self.gradients[f'b_{k+1}']=self.gradients[f'b_{k+1}']/self.batch_size

                        self.gradients[f'w_{k+1}'] =(self.gradients[f'w_{k+1}']) + self.weight_decay * self.parameters[f'w_{k+1}']
                        self.parameters[f'w_{k+1}'] -= self.learning_rate * (self.gradients[f'w_{k+1}'] )
                        self.parameters[f'b_{k+1}'] -= self.learning_rate * (self.gradients[f'b_{k+1}'] )

                pred_labels = self.feed_forwards(self.train_data)
                tra_acc = self.accuracy(pred_labels,self.train_labels)
                tra_loss = self.loss_act_func.loss_functions(pred_labels, self.train_labels)

                pred_labels = self.feed_forwards(self.val_data)
                val_acc = self.accuracy(pred_labels,self.val_labels)
                val_loss = self.loss_act_func.loss_functions(pred_labels, self.val_labels)
                print(f"Epoch {i+1}, Training Loss: {tra_loss} , Training Accuracy: {tra_acc} , Validation Loss: {val_loss} Validation Accuracy: {val_acc}")
                wandb.log({"Epoch": i+1, "Training_Loss": tra_loss, "Training_Accuracy": tra_acc,"Validation_Loss": val_loss,"Validation_Accuracy": val_acc})

        if(self.optimizer=="momentum"):
          '''
          Epoch 10, Loss: 0.6812374987038496 Acc: 75.0
          TEST LOSS: 0.7075 ACCURACY: 72.0900
          '''
          velocities={}
          for i in range(len(self.layer_sizes) - 1):
            velocities[f'w_{i+1}'] = np.zeros((self.layer_sizes[i], self.layer_sizes[i+1]))
            velocities[f'b_{i+1}'] = np.zeros((1, self.layer_sizes[i+1]))

          for i in range(self.epochs):
            for j in range(0,self.no_of_samples,self.batch_size):
              start = j
              end = start + self.batch_size
              Y_train_labels = self.train_labels[start:end]
              X_train_images = self.train_data[start:end]

              pred_labels=self.feed_forwards(X_train_images)
              self.back_prop(X_train_images, Y_train_labels,pred_labels)

              for k in range(len(self.layer_sizes) - 1):
                  self.gradients[f'w_{k+1}']=self.gradients[f'w_{k+1}']/self.batch_size
                  self.gradients[f'b_{k+1}']=self.gradients[f'b_{k+1}']/self.batch_size

                  self.gradients[f'w_{k+1}'] =(self.gradients[f'w_{k+1}']) + self.weight_decay * self.parameters[f'w_{k+1}']
                  # Update velocities with momentum
                  velocities[f'w_{k+1}'] = self.momentum * velocities[f'w_{k+1}'] + self.learning_rate * (self.gradients[f'w_{k+1}'])
                  velocities[f'b_{k+1}'] = self.momentum * velocities[f'b_{k+1}'] + self.learning_rate * (self.gradients[f'b_{k+1}'])

                  # Update parameters with momentum
                  self.parameters[f'w_{k+1}'] -= velocities[f'w_{k+1}']
                  self.parameters[f'b_{k+1}'] -= velocities[f'b_{k+1}']

            pred_labels = self.feed_forwards(self.train_data)
            tra_acc = self.accuracy(pred_labels,self.train_labels)
            tra_loss = self.loss_act_func.loss_functions(pred_labels, self.train_labels)

            pred_labels = self.feed_forwards(self.val_data)
            val_acc = self.accuracy(pred_labels,self.val_labels)
            val_loss = self.loss_act_func.loss_functions(pred_labels, self.val_labels)
            print(f"Epoch {i+1}, Training Loss: {tra_loss} , Training Accuracy: {tra_acc} , Validation Loss: {val_loss} Validation Accuracy: {val_acc}")
            wandb.log({"Epoch": i+1, "Training_Loss": tra_loss, "Training_Accuracy": tra_acc,"Validation_Loss": val_loss,"Validation_Accuracy": val_acc})

        if(self.optimizer=="nag"):
            '''
            Epoch 100, Loss: 0.8561739933889985 Acc: 63.355
            TEST LOSS: 0.8864 ACCURACY: 62.7400

            Epoch 20, Loss: 1.013375573148102 Acc: 61.795
            TEST LOSS: 1.0499 ACCURACY: 60.2300
            '''
           # Initialize velocities for weights and biases
            velocities={}
            for i in range(len(self.layer_sizes) - 1):
                velocities[f'w_{i+1}'] = np.zeros((self.layer_sizes[i], self.layer_sizes[i+1]))
                velocities[f'b_{i+1}'] = np.zeros((1, self.layer_sizes[i+1]))

            for i in range(self.epochs):
                for j in range(0,self.no_of_samples,self.batch_size):
                    start = j
                    end = start + self.batch_size
                    Y_train_labels = self.train_labels[start:end]
                    X_train_images = self.train_data[start:end]

                    '''
                    storing old weights in self.parameters[f'w_{k+1}_orig']
                    so that i can use it later and
                    updating self.parameters[f'w_{k+1}'] so that same feed forward function can be used.
                    '''
                    for k in range(len(self.layer_sizes) - 1):
                        self.parameters[f'w_{k+1}_orig']=self.parameters[f'w_{k+1}']
                        self.parameters[f'b_{k+1}_orig']=self.parameters[f'b_{k+1}']

                        self.parameters[f'w_{k+1}'] = self.parameters[f'w_{k+1}'] - self.momentum * velocities[f'w_{k+1}']
                        self.parameters[f'b_{k+1}'] = self.parameters[f'b_{k+1}'] - self.momentum * velocities[f'b_{k+1}']

                    #feed forward and backprop is applied after changing weights and biases
                    pred_labels=self.feed_forwards(X_train_images)
                    self.back_prop(X_train_images, Y_train_labels,pred_labels)

                    '''
                    here 1st we will set original weights to self.parameters[f'w_{k+1}']
                    and then update weigths and biases.
                    '''
                    for k in range(len(self.layer_sizes) - 1):

                        self.gradients[f'w_{k+1}']=self.gradients[f'w_{k+1}']/self.batch_size
                        self.gradients[f'b_{k+1}']=self.gradients[f'b_{k+1}']/self.batch_size

                        self.gradients[f'w_{k+1}'] =(self.gradients[f'w_{k+1}']) + self.weight_decay * self.parameters[f'w_{k+1}']

                        velocities[f'w_{k+1}'] = self.momentum * velocities[f'w_{k+1}'] + self.learning_rate * (self.gradients[f'w_{k+1}'] )
                        velocities[f'b_{k+1}'] = self.momentum * velocities[f'b_{k+1}'] + self.learning_rate * (self.gradients[f'b_{k+1}'] )

                        self.parameters[f'w_{k+1}']=self.parameters[f'w_{k+1}_orig']
                        self.parameters[f'b_{k+1}']=self.parameters[f'b_{k+1}_orig']

                        self.parameters[f'w_{k+1}'] -= velocities[f'w_{k+1}']
                        self.parameters[f'b_{k+1}'] -= velocities[f'b_{k+1}']

                pred_labels = self.feed_forwards(self.train_data)
                tra_acc = self.accuracy(pred_labels,self.train_labels)
                tra_loss = self.loss_act_func.loss_functions(pred_labels, self.train_labels)

                pred_labels = self.feed_forwards(self.val_data)
                val_acc = self.accuracy(pred_labels,self.val_labels)
                val_loss = self.loss_act_func.loss_functions(pred_labels, self.val_labels)
                print(f"Epoch {i+1}, Training Loss: {tra_loss} , Training Accuracy: {tra_acc} , Validation Loss: {val_loss} Validation Accuracy: {val_acc}")
                wandb.log({"Epoch": i+1, "Training_Loss": tra_loss, "Training_Accuracy": tra_acc,"Validation_Loss": val_loss,"Validation_Accuracy": val_acc})

        if(self.optimizer=="rmsprop"):
          '''
          Epoch 10, Loss: 0.6455246819185498 Acc: 75.41499999999999
          TEST LOSS: 0.6710 ACCURACY: 74.4000
          '''
          squared_gradients={}
          for i in range(len(self.layer_sizes) - 1):
              squared_gradients[f'w_{i+1}'] = np.zeros((self.layer_sizes[i], self.layer_sizes[i+1]))
              squared_gradients[f'b_{i+1}'] = np.zeros((1, self.layer_sizes[i+1]))

          for i in range(self.epochs):
              for j in range(0,self.no_of_samples,self.batch_size):
                  start = j
                  end = start + self.batch_size
                  Y_train_labels = self.train_labels[start:end]
                  X_train_images = self.train_data[start:end]

                  pred_labels=self.feed_forwards(X_train_images)
                  self.back_prop(X_train_images, Y_train_labels,pred_labels)

                  for k in range(len(self.layer_sizes) - 1):

                      self.gradients[f'w_{k+1}']=self.gradients[f'w_{k+1}']/self.batch_size
                      self.gradients[f'b_{k+1}']=self.gradients[f'b_{k+1}']/self.batch_size

                      self.gradients[f'w_{k+1}'] =(self.gradients[f'w_{k+1}']) + self.weight_decay * self.parameters[f'w_{k+1}']

                      # Calculate the squared gradients
                      squared_gradients[f'w_{k+1}'] = self.beta * squared_gradients[f'w_{k+1}'] + (1 - self.beta) * ((self.gradients[f'w_{k+1}']) ** 2)
                      squared_gradients[f'b_{k+1}'] = self.beta * squared_gradients[f'b_{k+1}'] + (1 - self.beta) * ((self.gradients[f'b_{k+1}']) ** 2)

                      # Update parameters using RMSprop
                      self.parameters[f'w_{k+1}'] -= (self.learning_rate * self.gradients[f'w_{k+1}'] / (np.sqrt(squared_gradients[f'w_{k+1}']) + self.epsilon))
                      self.parameters[f'b_{k+1}'] -= (self.learning_rate * self.gradients[f'b_{k+1}'] / (np.sqrt(squared_gradients[f'b_{k+1}']) + self.epsilon))

              pred_labels = self.feed_forwards(self.train_data)
              tra_acc = self.accuracy(pred_labels,self.train_labels)
              tra_loss = self.loss_act_func.loss_functions(pred_labels, self.train_labels)

              pred_labels = self.feed_forwards(self.val_data)
              val_acc = self.accuracy(pred_labels,self.val_labels)
              val_loss = self.loss_act_func.loss_functions(pred_labels, self.val_labels)
              print(f"Epoch {i+1}, Training Loss: {tra_loss} , Training Accuracy: {tra_acc} , Validation Loss: {val_loss} Validation Accuracy: {val_acc}")
              wandb.log({"Epoch": i+1, "Training_Loss": tra_loss, "Training_Accuracy": tra_acc,"Validation_Loss": val_loss,"Validation_Accuracy": val_acc})

        if(self.optimizer=="adam"):
          '''
          Epoch 10, Loss: 0.6223362734184407 Acc: 77.985
          TEST LOSS: 0.6613 ACCURACY: 76.2700
          '''
          moments={}
          for i in range(len(self.layer_sizes) - 1):
              moments[f'm_w_{i+1}'] = np.zeros((self.layer_sizes[i], self.layer_sizes[i+1]))
              moments[f'm_b_{i+1}'] = np.zeros((1, self.layer_sizes[i+1]))

          velocities={}
          for i in range(len(self.layer_sizes) - 1):
              velocities[f'v_w_{i+1}'] = np.zeros((self.layer_sizes[i], self.layer_sizes[i+1]))
              velocities[f'v_b_{i+1}'] = np.zeros((1, self.layer_sizes[i+1]))

          # t=0
          for i in range(self.epochs):
            t=i+1
            for j in range(0,self.no_of_samples,self.batch_size):
                # t=t+1
                start = j
                end = start + self.batch_size
                Y_train_labels = self.train_labels[start:end]
                X_train_images = self.train_data[start:end]

                pred_labels=self.feed_forwards(X_train_images)
                self.back_prop(X_train_images, Y_train_labels,pred_labels)
                # t=1
                for k in range(len(self.layer_sizes) - 1):

                    self.gradients[f'w_{k+1}'] = self.gradients[f'w_{k+1}'] / self.batch_size
                    self.gradients[f'b_{k+1}']=self.gradients[f'b_{k+1}']/self.batch_size

                    self.gradients[f'w_{k+1}'] = self.gradients[f'w_{k+1}'] + self.weight_decay * self.parameters[f'w_{k+1}']

                    # Update moments and velocities with Adam
                    moments[f'm_w_{k+1}'] = self.beta1 * moments[f'm_w_{k+1}'] + (1 - self.beta1) * self.gradients[f'w_{k+1}']
                    moments[f'm_b_{k+1}'] = self.beta1 * moments[f'm_b_{k+1}'] + (1 - self.beta1) * self.gradients[f'b_{k+1}']

                    velocities[f'v_w_{k+1}'] = self.beta2 * velocities[f'v_w_{k+1}'] + (1 - self.beta2) * (self.gradients[f'w_{k+1}'] ** 2)
                    velocities[f'v_b_{k+1}'] = self.beta2 * velocities[f'v_b_{k+1}'] + (1 - self.beta2) * (self.gradients[f'b_{k+1}'] ** 2)

                    # Bias correction
                    m_w_hat = moments[f'm_w_{k+1}'] / (1 - self.beta1 ** t)
                    m_b_hat = moments[f'm_b_{k+1}'] / (1 - self.beta1 ** t)

                    v_w_hat = velocities[f'v_w_{k+1}'] / (1 - self.beta2 ** t)
                    v_b_hat = velocities[f'v_b_{k+1}'] / (1 - self.beta2 ** t)

                    # Update parameters with Adam
                    self.parameters[f'w_{k+1}'] -= self.learning_rate * m_w_hat / (np.sqrt(v_w_hat) + self.epsilon)
                    self.parameters[f'b_{k+1}'] -= self.learning_rate * m_b_hat / (np.sqrt(v_b_hat) + self.epsilon)

                    # t=t+1

            pred_labels = self.feed_forwards(self.train_data)
            tra_acc = self.accuracy(pred_labels,self.train_labels)
            tra_loss = self.loss_act_func.loss_functions(pred_labels, self.train_labels)

            pred_labels = self.feed_forwards(self.val_data)
            val_acc = self.accuracy(pred_labels,self.val_labels)
            val_loss = self.loss_act_func.loss_functions(pred_labels, self.val_labels)
            print(f"Epoch {i+1}, Training Loss: {tra_loss} , Training Accuracy: {tra_acc} , Validation Loss: {val_loss} Validation Accuracy: {val_acc}")
            wandb.log({"Epoch": i+1, "Training_Loss": tra_loss, "Training_Accuracy": tra_acc,"Validation_Loss": val_loss,"Validation_Accuracy": val_acc})

        if(self.optimizer=="nadam"):
          '''
          Epoch 10, Loss: 0.645183860082157 Acc: 78.36333333333333
          TEST LOSS: 0.6794 ACCURACY: 77.5300
          '''
          moments={}
          for i in range(len(self.layer_sizes) - 1):
              moments[f'm_w_{i+1}'] = np.zeros((self.layer_sizes[i], self.layer_sizes[i+1]))
              moments[f'm_b_{i+1}'] = np.zeros((1, self.layer_sizes[i+1]))

          velocities={}
          for i in range(len(self.layer_sizes) - 1):
              velocities[f'v_w_{i+1}'] = np.zeros((self.layer_sizes[i], self.layer_sizes[i+1]))
              velocities[f'v_b_{i+1}'] = np.zeros((1, self.layer_sizes[i+1]))

          # t=0
          for i in range(self.epochs):
            t=1+i
            for j in range(0,self.no_of_samples,self.batch_size):
              # t=t+1
              start = j
              end = start + self.batch_size
              Y_train_labels = self.train_labels[start:end]
              X_train_images = self.train_data[start:end]

              pred_labels=self.feed_forwards(X_train_images)
              self.back_prop(X_train_images, Y_train_labels,pred_labels)

              for k in range(len(self.layer_sizes) - 1):

                  self.gradients[f'w_{k+1}'] = self.gradients[f'w_{k+1}'] / self.batch_size
                  self.gradients[f'b_{k+1}']=self.gradients[f'b_{k+1}']/self.batch_size

                  self.gradients[f'w_{k+1}'] = self.gradients[f'w_{k+1}'] + self.weight_decay * self.parameters[f'w_{k+1}']

                  # Update moments and velocities with Adam
                  moments[f'm_w_{k+1}'] = self.beta1 * moments[f'm_w_{k+1}'] + (1 - self.beta1) * self.gradients[f'w_{k+1}']
                  moments[f'm_b_{k+1}'] = self.beta1 * moments[f'm_b_{k+1}'] + (1 - self.beta1) * self.gradients[f'b_{k+1}']

                  velocities[f'v_w_{k+1}'] = self.beta2 * velocities[f'v_w_{k+1}'] + (1 - self.beta2) * (self.gradients[f'w_{k+1}'] ** 2)
                  velocities[f'v_b_{k+1}'] = self.beta2 * velocities[f'v_b_{k+1}'] + (1 - self.beta2) * (self.gradients[f'b_{k+1}'] ** 2)

                  # Bias correction
                  m_w_hat = moments[f'm_w_{k+1}'] / (1 - self.beta1 ** t)
                  m_b_hat = moments[f'm_b_{k+1}'] / (1 - self.beta1 ** t)

                  v_w_hat = velocities[f'v_w_{k+1}'] / (1 - self.beta2 ** t)
                  v_b_hat = velocities[f'v_b_{k+1}'] / (1 - self.beta2 ** t)

                  # Update parameters with NAdam
                  nadam_factor = (1 - self.beta1) / (1 - (self.beta1 ** t))
                  weigth_update = nadam_factor * self.gradients[f'w_{k+1}'] + (self.beta1 * m_w_hat)
                  bias_update = nadam_factor * self.gradients[f'b_{k+1}'] + (self.beta1 * m_b_hat)
                  self.parameters[f'w_{k+1}'] -= ((self.learning_rate / (np.sqrt(v_w_hat) + self.epsilon)) * weigth_update )
                  self.parameters[f'b_{k+1}'] -= ((self.learning_rate / (np.sqrt(v_b_hat) + self.epsilon)) * bias_update )

              # t=t+1

            pred_labels = self.feed_forwards(self.train_data)
            tra_acc = self.accuracy(pred_labels,self.train_labels)
            tra_loss = self.loss_act_func.loss_functions(pred_labels, self.train_labels)

            pred_labels = self.feed_forwards(self.val_data)
            val_acc = self.accuracy(pred_labels,self.val_labels)
            val_loss = self.loss_act_func.loss_functions(pred_labels, self.val_labels)
            print(f"Epoch {i+1}, Training Loss: {tra_loss} , Training Accuracy: {tra_acc} , Validation Loss: {val_loss} Validation Accuracy: {val_acc}")
            wandb.log({"Epoch": i+1, "Training_Loss": tra_loss, "Training_Accuracy": tra_acc,"Validation_Loss": val_loss,"Validation_Accuracy": val_acc})

    def accuracy(self, pred, truth):
      return ((np.argmax(truth, axis=1) == np.argmax(pred, axis=1)).mean())*100

    def test(self, x, y):
      pred_labels=self.feed_forwards(x)
      loss = self.loss_act_func.loss_functions(pred_labels, y)
      acc = self.accuracy(pred_labels, y)
      print(f'TEST LOSS: {loss:.4f} ACCURACY: {acc:.4f}')


# loss and activation

In [ ]:
class LossActFunc():
    def __init__(self, lossFun = "cross_entropy",actFun="sigmoid"):
        self.loss_func = lossFun
        self.act_fun = actFun

    def loss_functions(self,pred, actual):
      if (self.loss_func=="cross_entropy"):
        return -np.sum(actual * np.log(pred + 1e-9)) / len(pred)

      elif (self.loss_func == "mean_squared_error"):
        squErr=(1/2)* np.sum((pred - actual) ** 2)
        meanSquErr=squErr/ len(pred)
        return  meanSquErr

    def get_name(self):
      if (self.loss_func=="cross_entropy"):
        return "cross_entropy"

      elif (self.loss_func == "mean_squared_error"):
        return "mean_squared_error"


    def activation_func(self, x):

      if(self.act_fun == "sigmoid"):
        # sigmoid function
        # print("in sigmoid",x)
        return 1.0 / (1.0 + np.exp(-x))

      elif(self.act_fun == "tanh"):
          # tanh function
          return np.tanh(x)

      elif(self.act_fun == "ReLU"):
        return np.maximum(0, x)

    def derivation_of_activation(self, x):

      if(self.act_fun == "sigmoid"):
        sigmoid_x = self.activation_func(x)
        return sigmoid_x * (1 - sigmoid_x)

      if(self.act_fun == "tanh"):
          return 1 - (np.tanh(x) ** 2)

      if(self.act_fun == "ReLU"):
        return (x > 0) * 1


In [ ]:
# class ActivationFunctions():
#     def __init__(self, fun = "sigmoid"):
#         self.act_fun = fun

#     def activation_func(self, x):

#         if(self.act_fun == "sigmoid"):
#           # sigmoid function
#           # print("in sigmoid",x)
#           return 1.0 / (1.0 + np.exp(-x))

#         elif(self.act_fun == "tanh"):
#             # tanh function
#             return np.tanh(x)

#         elif(self.act_fun == "ReLU"):
#           return np.maximum(0, x)

#     def derivation_of_activation(self, x):

#         if(self.act_fun == "sigmoid"):
#           sigmoid_x = self.activation_func(x)
#           return sigmoid_x * (1 - sigmoid_x)

#         if(self.act_fun == "tanh"):
#             return 1 - (np.tanh(x) ** 2)

#         if(self.act_fun == "ReLU"):
#           return (x > 0) * 1

# Load & Preprocess

In [ ]:
#Load dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
total_class=max(train_labels)

train_images = train_images.reshape(train_images.shape[0], -1) / 255.0
test_images = test_images.reshape(test_images.shape[0], -1) / 255.0

#change trainlabel=2 to oneHot=[0 0 1 0 0 0 0 0 0 0]
train_labels_one_hot = np.eye(10)[train_labels]
test_labels_one_hot = np.eye(10)[test_labels]

train_images, val_images, train_labels_one_hot, val_labels_one_hot = train_test_split(train_images, train_labels_one_hot, test_size=0.1, random_state=42)
# print(len(train_images))
# print(len(val_images))



# Set param and call

In [ ]:
layer_sizes = [train_images.shape[1],256,512,10]  # Input, hidden1, hidden2,hidden3 output

# gent Starting Run: dlaobhoi with config:
# wandb: 	activation: ReLU
# wandb: 	batch_size: 64
# wandb: 	epochs: 5
# wandb: 	hidden_layers: [128, 128, 128, 128, 128]
# wandb: 	initialization: random
# wandb: 	learning_rate: 0.001
# wandb: 	optimizer: momentum
# wandb: 	weight_decay: 0.5

loss_act_func = LossActFunc("cross_entropy","ReLU") #"mean_squared_error", "cross_entropy" # sigmoid, ReLU, tanh
# loss_func = LossFunc("cross_entropy")
epochs=5
initialization="Xavier" #random , Xavier
learning_rate=0.0005
momentum=0.9  #for momentum and nag
beta=0.5     #for rmsprop
beta1=0.9    #for adam and nadam
beta2=0.999    #for adam and nadam
epsilon=1e-8
weight_decay=0
batch_size=32
optimizer="nadam" # sgd , momentum ,nag , rmsprop , adam , nadam

# Epoch 50, Training Loss: 0.09106023231455047 , Training Accuracy: 96.66481481481482 , Validation Loss: 0.5375967565066632 Validation Accuracy: 88.78333333333333
# TEST LOSS: 0.5613 ACCURACY: 88.5200

mlp = MultiLayerPerceptron(train_images, train_labels_one_hot,val_images,val_labels_one_hot, layer_sizes,initialization,loss_act_func,batch_size,epochs,learning_rate,weight_decay,optimizer,momentum,beta,beta1,beta2,epsilon)
mlp.init_weights()
mlp.optimizer_func()
mlp.test(test_images, test_labels_one_hot)

# question4 sweep

In [ ]:
sweep_config = {
    # RandomSearch over the hyperparameters
    "method": "random",
    "name":"Random_sweep",
    "metric": {"goal": "maximize", "name": "validationaccuracy"},
    "parameters": {
        "activation": {"values": ["sigmoid", "tanh", "ReLU"]},
        "batch_size": {"values": [32, 64, 128]},
        "optimizer": {"values": ["sgd", "momentum", "nag", "rmsprop", "adam", "nadam"]},
        "no_of_hidden_layer": {"values": [3,4,5]},
        "size_of_hidden_layerr": {"values": [256,128,64,32]},
        "epochs": {"values": [5, 10, 30]},
        "weight_decay": {"values": [0.0, 0.0005, 0.5]},
        "learning_rate": {"values": [1e-3, 1e-4]},
        "initialization": {"values": ["Xavier", "random"]}
    }
}

In [ ]:
def train():
   init_sweep =  wandb.init(project="CS6910 - Assignment 1", name="Question4")
   sweep_params = init_sweep.config

   wandb.run.name = "_ac_" + sweep_params.activation + "_bs_" + str(sweep_params.batch_size) + "_op_" + sweep_params.optimizer + "_hl_" + str(sweep_params.no_of_hidden_layer)+ "_neu_" + str(sweep_params.size_of_hidden_layerr) +"_ep_" + str(sweep_params.epochs)+"_wd_" + str(sweep_params.weight_decay)+"_lr_" + str(sweep_params.learning_rate)+"_ini_"+sweep_params.initialization

   loss_act_func = LossActFunc("cross_entropy","ReLU")
   epochs=sweep_params.epochs
   initialization=sweep_params.initialization #random , Xavier
   learning_rate=sweep_params.learning_rate
   momentum=0.9  #for momentum and nag
   beta=0.9     #for rmsprop
   beta1=0.9    #for adam and nadam
   beta2=0.999    #for adam and nadam
   epsilon=0.000001
   weight_decay=sweep_params.weight_decay
   batch_size=sweep_params.batch_size
   optimizer=sweep_params.optimizer # sgd , momentum ,nag , rmsprop , adam , nadam

   # layer_sizes = [train_images.shape[1],5,10,15,10]
   layer_sizes=[]
   layer_sizes.append(train_images.shape[1])
   for i in range(sweep_params.no_of_hidden_layer):
      layer_sizes.append(sweep_params.size_of_hidden_layerr)
   layer_sizes.append(10)

   print("layer_sizes=",layer_sizes)

   mlp = MultiLayerPerceptron(train_images, train_labels_one_hot,val_images,val_labels_one_hot, layer_sizes,initialization,loss_act_func,batch_size,epochs,learning_rate,weight_decay,optimizer,momentum,beta,beta1,beta2,epsilon)
   mlp.init_weights()
   mlp.optimizer_func()
   y_pred = mlp.feed_forwards(val_images)

   val_accuracy=mlp.accuracy(y_pred,val_labels_one_hot)
   wandb.log({"validationaccuracy": val_accuracy})
   # mlp.test(test_images, test_labels_one_hot)
   #  print("test_acc=",test_acc)


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="CS6910 - Assignment 1")
wandb.agent(sweep_id, train, count=100)
wandb.finish()

# Q4_bayesian_sweep

In [ ]:
sweep_config = {
    # RandomSearch over the hyperparameters
    "name" : "Bayesian Sweep",
    "method": "bayes",
    "metric": {"goal": "maximize", "name": "validationaccuracy"},
    "parameters": {
        "activation": {"values": ["sigmoid", "tanh", "ReLU"]},
        "batch_size": {"values": [32, 64, 128]},
        "optimizer": {"values": ["sgd", "momentum", "nag", "rmsprop", "adam", "nadam"]},
        "no_of_hidden_layer": {"values": [3,4,5]},
        "size_of_hidden_layerr": {"values": [256,128,64,32]},
        "epochs": {"values": [5, 10, 30]},
        "weight_decay": {"values": [0.0, 0.0005, 0.5]},
        "learning_rate": {"values": [1e-3, 1e-4]},
        "initialization": {"values": ["Xavier", "random"]}
    }
}

In [ ]:
def train():
   init_sweep =  wandb.init(project="CS6910 - Assignment 1", name="Question4_bayes")
   sweep_params = init_sweep.config

   wandb.run.name = "_ac_" + sweep_params.activation + "_bs_" + str(sweep_params.batch_size) + "_op_" + sweep_params.optimizer +"_hl_" + str(sweep_params.no_of_hidden_layer)+ "_neu_" + str(sweep_params.size_of_hidden_layerr) +"_ep_" + str(sweep_params.epochs)+"_wd_" + str(sweep_params.weight_decay)+"_lr_" + str(sweep_params.learning_rate)+"_ini_"+sweep_params.initialization

   loss_act_func = LossActFunc("cross_entropy","ReLU")
    # sigmoid, ReLU, tanh
    #"mean_squared_error", "cross_entropy"
   epochs=sweep_params.epochs
   initialization=sweep_params.initialization #random , Xavier
   learning_rate=sweep_params.learning_rate
   momentum=0.9  #for momentum and nag
   beta=0.9     #for rmsprop
   beta1=0.9    #for adam and nadam
   beta2=0.999    #for adam and nadam
   epsilon=0.000001
   weight_decay=sweep_params.weight_decay
   batch_size=sweep_params.batch_size
   optimizer=sweep_params.optimizer # sgd , momentum ,nag , rmsprop , adam , nadam

   # layer_sizes = [train_images.shape[1],5,10,15,10]
   layer_sizes=[]
   layer_sizes.append(train_images.shape[1])
   for i in range(sweep_params.no_of_hidden_layer):
      layer_sizes.append(sweep_params.size_of_hidden_layerr)
   layer_sizes.append(10)

   print("layer_sizes=",layer_sizes)

   mlp = MultiLayerPerceptron(train_images, train_labels_one_hot,val_images,val_labels_one_hot, layer_sizes,initialization,loss_act_func,batch_size,epochs,learning_rate,weight_decay,optimizer,momentum,beta,beta1,beta2,epsilon)
   mlp.init_weights()
   mlp.optimizer_func()
   y_pred = mlp.feed_forwards(val_images)

   val_accuracy=mlp.accuracy(y_pred,val_labels_one_hot)
   wandb.log({"validationaccuracy": val_accuracy})
   # mlp.test(test_images, test_labels_one_hot)
   #  print("test_acc=",test_acc)


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="CS6910 - Assignment 1")
wandb.agent(sweep_id, train, count=100)
wandb.finish()

# question7

In [ ]:

#set best params
layer_sizes = [train_images.shape[1],256,256,256,128,128,10]  # Input, hidden1, hidden2,hidden3 output

act_func = ActivationFunctions("tanh") # sigmoid, ReLU, tanh
loss_func = LossFunc("cross_entropy") #"mean_squared_error", "cross_entropy"
epochs=30
initialization="Xavier" #random , Xavier
learning_rate=0.0001
momentum=0.9  #for momentum and nag
beta=0.5     #for rmsprop
beta1=0.9    #for adam and nadam
beta2=0.999    #for adam and nadam
epsilon=1e-8
weight_decay=0
batch_size=32
optimizer="adam" # sgd , momentum ,nag , rmsprop , adam , nadam

# Epoch 50, Training Loss: 0.09106023231455047 , Training Accuracy: 96.66481481481482 , Validation Loss: 0.5375967565066632 Validation Accuracy: 88.78333333333333
# TEST LOSS: 0.5613 ACCURACY: 88.5200

mlp = MultiLayerPerceptron(train_images, train_labels_one_hot,val_images,val_labels_one_hot, layer_sizes,initialization,act_func,loss_func,batch_size,epochs,learning_rate,weight_decay,optimizer,momentum,beta,beta1,beta2,epsilon)
mlp.init_weights()
mlp.optimizer_func()
mlp.test(test_images, test_labels_one_hot)

In [ ]:
className_images = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

pred_class = mlp.feed_forwards(test_images)

wandb.init(project="CS6910 - Assignment 1", name="Question7")

# # print(len(pred_class))
# print(np.argmax(pred_class,1)[0])
# print(np.argmax(pred_class[0]))

mat_predicted=[]
mat_true=[]
for i in range(len(pred_class)):
  mat_predicted.append(np.argmax(pred_class[i]))
for i in range(len(pred_class)):
  mat_true.append(np.argmax(test_labels_one_hot[i]))

fig, ax = plt.subplots(figsize=(10, 8))
confusion_matrix = metrics.confusion_matrix(mat_true , mat_predicted)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = className_images)
plt.title('Confusion Matrix Fashion', fontsize=10)
cm_display.plot(ax=ax)
# plt.show()

wandb.log({"Confusion_Matrix": wandb.Image(plt)})
wandb.finish()

In [ ]:
wandb.finish()